# GelPy demo

## Statement of need

Quantitative gel analysis has been an intricate and time-consuming task for life scientists, often hindering prompt data-driven decisions. This Python package, designed for rapid, simple, and reproducible extraction of quantitative data from gel electrophoresis images, could significantly streamline this process. By lowering barriers to performing gel analysis, it empowers more researchers to exploit this method effectively, enriching the scientific discourse.

## Outline of this notebook

1. Introduction/Motivation
* Reproducability
* Fiji works, but is painfull
* If one wants to work with the quantitative data afetrwards, e.g. for some plots, one has to export the file from e.g. Fiji and read it in again in e.g. python. -> way better to have all of the interesting data as python objects.
* It can be hard to discriminate two bands, which are close together, using hard integration cutoffs, potentially introducing errors. A fitting process does better.

2. Live demonstration

* Load an image and describe valid input images
* setting up a gel
* removing background with some explaination what is done
* show and save adjusted images
    * If all you need is to have a labelled and contrast adjusted image, then here you go, no more powerpoint or such needed (ladder labels coming soon). BUT, I'd like to convince you that with this package, the quantitative data is just a minute away.
* show selection of lanes by name or index
* show fitting with gaussian or emg of e.g. ladders or a merged peak.
* show the "extraction" of the data from the gel object and use it to do some further computations (e.g. plot folding yield of origami). Or normalizing the intensities to a specific band or something.
* report example issue on github

3. Discussion of benefits, potential improvements and issues

* Peak selection is still parameter sensitive, meaning that one has to tweak some parameters if too many or not enough peaks are recognized for fitting.
* There is still no function to recognize and label DNA/RNA/protein ladders. 
* Still using a single processor, so the fitting scales with the number of bands* the number of lanes linearly (xx seconds per band on my machine)

4. Call for others to test, report issues on github, contribute and share
* I tested it on images I had lying around, it will likely have issues when you try it on different cases. I am actually hoping it does, and I am hoping even more that you are motivated to report them as issues on github so I can work on them, or, if you are also keen to contribute to your first python package, just fix it yourself and submit a merge request.

In [ ]:
from gelpy import AgaroseGel

gel = AgaroseGel("data/20230317_first_fold_cy3_500v-[Cy3]-1.tif")
gel.setup_gel(labels=['1kb ladder', 'scaf', '0b', '0b', '0b', '4b', '4b', '4b', '6b', '6b', '6b'], x_label_pos=(120, 1910, 11),
              line_profile_width=100)
gel.remove_background(model="2d_plane_fit")

In [ ]:
gel.show_adjusted_images(show_type="linear")
gel.show_line_profiles(select_lanes=[0,1],
                       slice_line_profile_length=(100, 900),
                       fit="gaussian",
                       maxima_prominence=0.0005, maxima_threshold=0.001,
                       plot_fits=True)